In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-07-29 12:23:59.176314: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
#load images from folder
#augment images with transforms here :)
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_generator = train_datagen.flow_from_directory("./training",target_size=(300,300),class_mode='binary')

#validation data, used while training
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
    "./validation",
    target_size=(300,300),
    class_mode='binary',
)

#callback to stop training when 80% accuracy reached on validation set
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>0.80):
            print("\nAccuracy reached 80%, stopping training")
            self.model.stop_training = True
callbacks = myCallback()

Found 80 images belonging to 2 classes.
Found 20 images belonging to 2 classes.


In [68]:
#create model
model = tf.keras.models.Sequential([
    #a bunch of convolution and pooling layers to filter down the image to essential features (hopefully)
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    #dropout for regularization (this really impacted model performance)
    tf.keras.layers.Dropout(0.2),
    #sigmoid drives values towards 0 and 1 for binary classification problems
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 298, 298, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ (None, 149, 149, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 147, 147, 32)   │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 73, 73, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 71, 71, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 35, 35, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 33, 33, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │     1,606,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,704,097 (6.50 MB)

 Trainable params: 1,704,097 (6.50 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
#compile model with rmsprop optimizer
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [70]:
#train model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=callbacks)

Epoch 1/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5490 - loss: 0.6975 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 668ms/step - accuracy: 0.4195 - loss: 0.6959 - val_accuracy: 0.7000 - val_loss: 0.6901
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 663ms/step - accuracy: 0.6609 - loss: 0.6905 - val_accuracy: 0.5000 - val_loss: 0.6909
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 838ms/step - accuracy: 0.5490 - loss: 0.7626 - val_accuracy: 0.5000 - val_loss: 0.6919
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 656ms/step - accuracy: 0.5188 - loss: 0.6913 - val_accuracy: 0.6500 - val_loss: 0.6875
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 680ms/step - accuracy: 0.6547 - loss: 0.6880 - val_accuracy: 0.5000 - val_loss: 0.6851
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 669ms/step - accuracy: 0.5464 - loss: 0.6817 - val_accuracy: 0.7000 - val_loss: 0.6688
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.5500 - loss: 0.6832
Accuracy reached 85%, stopping traini

In [141]:
#test on custom images
img = image.load_img("./custom/visible/2.png", target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)

print(train_generator.class_indices)
print(classes)
if classes[0]>0.5:
    print("is visible")
else:
    print("is not visible")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
{'not_visible': 0, 'visible': 1}
[[0.99999946]]
is visible


In [57]:
#Take video input and use the model to determine if an intruder exists

#intruder = False
#video -> frames -> model(frame)
#if model(frame) == visible:
#intruder = True
#else:
#continue
#return intruder

In [129]:
#save model to disk
model.save("SecurityModel.keras")

In [150]:
#load it and create a single function that takes a file and outputs a prediction
loaded_model = tf.keras.models.load_model('SecurityModelFinal.keras')

def classify(imgPath):
    #preprocess image into tensor
    img = image.load_img(imgPath, target_size=(300,300))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    tensor = np.vstack([x])
    #classify tensor
    classes = model.predict(tensor)
    if classes[0]>0.5:
        print("model prediction: is visible")
        return True
    else:
        print("model prediction: is not visible")
        return False


In [152]:
#use loaded_model
classify("custom/visible/3.png")
classify("custom/not_visible/5.png")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
model prediction: is visible
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
model prediction: is not visible


False